In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from datetime import datetime
from loguru import logger
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.object_quantification.controller import QuantificationController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\sdata_IDC-N-P\analysis_settings_IDC-N-P.yaml'

settings = load_analysis_settings(settings_path)
settings

2025-11-14 11:26:56.614 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/IDC-N-P/IDC-N-P_Final', analysis_name='IDC-N-P_Analysis', local_analysis_dir='C:/sdata_IDC-N-P', remote_analysis_dir='', log_dir=None), core_detection=CoreDetectionSettings(detection_image='IDC-N-P_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), InstansegStep(categ

### Define the logger

In [3]:
log_file = settings.log_dir_path / f"quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for the analysis

In [4]:
core_dir = settings.analysis_dir / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/sdata_IDC-N-P/IDC-N-P_Analysis/cores/Core_000.zarr'),
 WindowsPath('C:/sdata_IDC-N-P/IDC-N-P_Analysis/cores/Core_001.zarr')]

### Setup

In [5]:
# setup quantification controllers
quant_controller_list = [] 
qc_prefix = settings.qc.prefix
for quant in settings.quant:

    table_name = quant.name
    masks_keys = quant.masks
    mask_to_annotate = quant.layer_connection

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{mask_to_annotate}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        mask_to_annotate=mask_to_annotate,
        overwrite=True,
        quantify_qc = True,
        qc_prefix = qc_prefix,
    )

    quant_controller_list.append(controller) 

2025-11-14 11:27:05.003 | INFO     | __main__:<module>:10 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell', 'ring': 'ring', 'cyto': 'cytoplasm'} and connection to 'instanseg_cell' mask


### Quantify

In [6]:
for sd_path in path_list:
    
    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-14 11:27:08.174 | INFO     | __main__:<module>:4 - Processing Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-14 11:27:11.761 | INFO     | plex_pipe.object_quantification.controller:validate_sdata_as_input:229 - Channels not specified. Quantifying all existing channels (13).
2025-11-14 11:28:03.914 | INFO     | plex_pipe.object_quantification.controller:run:253 - Prepared masks for quantification.
2025-11-14 11:28:03.914 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'nucleus'
2025-11-14 11:28:28.502 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'cell'
2025-11-14 11:29:16.599 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'ring'
2025-11-14 11:29:46.718 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'cyto'
2025-11-14 11:30:35.865 | INFO     | plex_pipe.object_quantification.controller:run:265 - Found 4 columns with multiple dimens

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


...

2025-11-14 11:39:20.579 | SUCCESS  | plex_pipe.object_quantification.controller:run:338 - Quantification complete. Table 'instanseg_table' written to C:\sdata_IDC-N-P\IDC-N-P_Analysis\cores\Core_000.zarr
2025-11-14 11:39:20.590 | INFO     | __main__:<module>:4 - Processing Core_001.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-14 11:39:22.974 | INFO     | plex_pipe.object_quantification.controller:validate_sdata_as_input:224 - Quantifying 13 user-specified channels: ['AP2B', 'AR', 'CD45', 'CK14', 'CK818', 'DAPI', 'ECad', 'ER', 'FOXA1', 'GATA3', 'HER2', 'ProgRc', 'SMA'].
2025-11-14 11:39:31.216 | INFO     | plex_pipe.object_quantification.controller:run:253 - Prepared masks for quantification.
2025-11-14 11:39:31.216 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'nucleus'
2025-11-14 11:39:33.985 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'cell'
2025-11-14 11:39:38.872 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'ring'
2025-11-14 11:39:41.914 | INFO     | plex_pipe.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'cyto'
2025-11-14 11:39:46.410 | INFO     | plex_pi

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


2025-11-14 11:40:57.372 | INFO     | plex_pipe.object_quantification.controller:run:295 - Quantification complete. Resulting AnnData has 2796 observations and 105 variables.
2025-11-14 11:40:57.389 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:34 - Table instanseg_table present in the spatialdata object.
2025-11-14 11:40:57.389 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:43 - Centroids: centroid_cell present in the anndata table instanseg_table.
2025-11-14 11:40:58.557 | SUCCESS  | plex_pipe.object_quantification.controller:run:338 - Quantification complete. Table 'instanseg_table' written to C:\sdata_IDC-N-P\IDC-N-P_Analysis\cores\Core_001.zarr


### Sneak peek

In [7]:
sd_path = path_list[0]
sdata = sd.read_zarr(sd_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

In [8]:
from napari_spatialdata import Interactive

Interactive(sdata)

2025-11-14 11:42:16.255 | WARNING  | napari_spatialdata._viewer:__init__:57 - Due to Shift-L being used as shortcut in napari, it is being deprecated and might not link a new layer to an existing SpatialData object in the viewer. Please use ⌘-L on MacOS or else Ctrl-L.


INFO: Please wait for the current operation to finish.
2025-11-14 11:43:48.990 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-14 11:44:28.929 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-14 11:45:42.034 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-14 11:46:00.050 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-14 11:47:55.598 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-14 11:47:55.598 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-14 11:48:35.970 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
